In [ ]:
import json
from babyTLSUtil import *
from random import randint
from os import urandom
from key_exchange import *
import telnetlib
import json


KEP = KEY_EXCHANGE("ECDHKE", "secp384r1")
client_public_key = KEP.generate_public_key()

client_hello = {
    "Random": urandom(16).hex(),
    "Key_Share": client_public_key
}
print("Client Hello:", json.dumps(client_hello)) # send to server

HOST = "0.0.0.0"
PORT = 1337

tn = telnetlib.Telnet(HOST, PORT)


def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)


print(readline())
json_send(client_hello)
server_hello = json.loads(readline().decode()[38:])
server_public_key = server_hello["Key_Share"]
shared_secret = KEP.generate_shared_secret(server_public_key)

# hello verification
HASH_ALG = hashlib.sha384 # TODO: cipher suite negotiation
HASH_LEN = HASH_ALG().digest_size
client_hello = json.dumps(client_hello).encode()
server_hello = json.dumps(server_hello).encode()
hello_hash = HASH_ALG(client_hello + server_hello).digest()
result = hello_verify(hello_hash, shared_secret, HASH_ALG, HASH_LEN)
print(result)

server_certificate = json.loads(readline().decode()[20:-1])
server_certificate_verify = json.loads(readline().decode()[27:-1])
finished = json.loads(readline().decode()[27:-1])

# Master key calculation (encrypts application traffic)
key = result["server_handshake_key"]
iv = result["server_handshake_iv"]

cert = decrypt_and_verify(server_certificate["Encrypted_Data"], server_certificate["Record_Header"], key, iv, server_certificate["Auth_Tag"])
signature = decrypt_and_verify(server_certificate_verify["Encrypted_Data"], server_certificate_verify["Record_Header"], key, iv, server_certificate_verify["Auth_Tag"])
verify_data = decrypt_and_verify(finished["Encrypted_Data"], finished["Record_Header"], key, iv, finished["Auth_Tag"])
handshake_hash = HASH_ALG(client_hello + server_hello + cert + signature + verify_data).digest()
last_result = handshake_verify(handshake_hash, bytes.fromhex(result["handshake_secret"]), HASH_ALG, HASH_LEN)
print(last_result)

# finished_key = HKDF-Expand-Label(key: client_secret, label: "finished", ctx: "", len: 32)
finished_key = HKDF_expand_label(bytes.fromhex(result["client_secret"]), b"finished", b"", 32, HASH_ALG)
# finished_hash = SHA384(Client Hello ... Server Finished)
finished_hash = HASH_ALG(client_hello + server_hello + cert + signature + verify_data).digest()
# verify_data = HMAC-SHA384(key: finished_key, msg: finished_hash)
client_verify_data = tls_HMAC(finished_key, finished_hash, HASH_ALG)
header = "1703030045" # Record Header - CHANGE THIS: depends on the data follows
client_finished_encrypted, auth_tag = encrypt_and_digest(client_verify_data, header, key, iv)
client_finished = {
    "Record_Header": header,
    "Encrypted_Data": client_finished_encrypted.hex(),
    "Auth_Tag": auth_tag.hex()
}
json_send(client_finished)
print(client_finished)